In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-06-02 14:35:38.814872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 14:35:38.905518: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-02 14:35:39.228430: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 40

N_CHANNELS = 3
HEIGHT = 240
WIDTH = 320


SHOW_PREVIEW = True

FIXED_DELTA_SECONDS = 0.2

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    discrete_actions = {
    0: [0, 1], 1: [0, 0.5], 2: [0, 0]}
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.actor_list = []

        self.action_space = spaces.Discrete(len(self.discrete_actions))

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)

        self.client.load_world('Town01')

        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        self.settings.no_rendering_mode = False

        self.world.apply_settings(self.settings)

        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.cleanup()
        
        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        
        self.transform = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.transform.location
        self.point_b = self.spawn_points[241].location
        
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.transform)
        
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()
            return self.reset()
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")

        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z, x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2)  # not detect a collision when the car spawns/falls from sky.

        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     
        while self.front_camera is None:
            time.sleep(0.01)  # Just in case car takes any longer.

        self.episode_start = time.time()
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        # Show camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
        
    def step(self, action):
        self.step_counter += 1
        action = int(action) 
        steer, throttle = self.discrete_actions[action]

        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        self.vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer, brake=0.0))

        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        print(f" Throttle:{throttle}, Current speed: {kmh} km/h, Distance to target: {distance_to_destination} meters")

        # Decelerate as the vehicle approaches the target(Smooth Stopping)
        #distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        #if distance_to_destination < 50.0:  # Within 10 meters of the target
        #    throttle = max(0.1, throttle * (distance_to_destination / 50.0))  # never below 0.1 3shan mto2afsh 5ales abl ma nwsl
            

        
        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # Initialize variables
        terminated = False
        truncated = False
        
        # Calculate elapsed time since the episode started
        elapsed_time = time.time() - self.episode_start

        # Check for collision
        if len(self.collision_hist) != 0:
            print("Collision occurred")
            #reward = -5
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            #observation = self.reset()[0]

        # Check if the car has reached the goal point
        if distance_to_destination < 10.0:  # Threshold for reaching the destination
            print("Goal reached")
            reward = 5
            terminated = True
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))  # stop the car
            #observation = self.reset()[0]
        else:
            reward = kmh * (1 / distance_to_destination) * (1 / elapsed_time)  # Combined reward


        # Check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            print("Time limit exceeded")
            terminated = True
            truncated = True
            #self.cleanup()
            #observation = self.front_camera

        # Get observation
        observation = self.front_camera

        return observation.astype(np.uint8), reward, terminated, truncated, {}  # Return an empty dictionary as info

    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        cv2.destroyAllWindows()

    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3]  # Ignore the 4th Alpha channel - up to 3
        self.front_camera = i / 255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:

env = CarEnv()

check_env(env)


Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!


In [4]:
model_name = "three_throttle_one_steer"
models_dir = f"models/{model_name}/"

logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1, n_steps =12288, learning_rate=0.001, tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<8:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1716418565/PPO_0
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.17e+04 |
|    ep_rew_mean     | 320      |
| time/              |          |
|    fps             | 406      |
|    iterations      | 1        |
|    time_elapsed    | 30       |
|    total_timesteps | 12288    |
---------------------------------
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 7.88e+03     |
| 

Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.2e+03     |
|    ep_rew_mean          | 185         |
| time/                   |             |
|    fps                  | 166         |
|    iterations           | 10          |
|    time_elapsed         | 736         |
|    total_timesteps      | 122880      |
| train/                  |             |
|    approx_kl            | 0.002017664 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | 1.19e-07    |
|    learning_rate        | 0.001       |
|    loss                 | 0.0115      |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.000123   |
|    value_loss           | 0.00986     |
-------------

Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.9e+03     |
|    ep_rew_mean          | 167         |
| time/                   |             |
|    fps                  | 161         |
|    iterations           | 19          |
|    time_elapsed         | 1443        |
|    total_timesteps      | 233472      |
| train/                  |             |
|    approx_kl            | 0.007868278 |
|    clip_fraction        | 0.00291     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | -0.00584    |
|    n_updates            | 180         |
|    policy_gradient_loss | -0.000798   |
|    value_loss           | 0.00434     |
-----------------------------------------
Collision occurred
Time limit exceeded
Tr

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.53e+03    |
|    ep_rew_mean          | 169         |
| time/                   |             |
|    fps                  | 160         |
|    iterations           | 28          |
|    time_elapsed         | 2143        |
|    total_timesteps      | 344064      |
| train/                  |             |
|    approx_kl            | 0.002309202 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.08       |
|    explained_variance   | 1.19e-07    |
|    learning_rate        | 0.001       |
|    loss                 | 0.0441      |
|    n_updates            | 270         |
|    policy_gradient_loss | -0.000183   |
|    value_loss           | 0.238   

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.8e+03     |
|    ep_rew_mean          | 192         |
| time/                   |             |
|    fps                  | 158         |
|    iterations           | 36          |
|    time_elapsed         | 2788        |
|    total_timesteps      | 442368      |
| train/                  |             |
|    approx_kl            | 0.004240136 |
|    clip_fraction        | 0.00831     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.01       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.001       |
|    loss                 | 0.515       |
|    n_updates            | 350         |
|    policy_gradient_loss | -0.000993   |
|    value_loss           | 0.872   

...

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.07e+03      |
|    ep_rew_mean          | 676           |
| time/                   |               |
|    fps                  | 161           |
|    iterations           | 9             |
|    time_elapsed         | 683           |
|    total_timesteps      | 3637248       |
| train/                  |               |
|    approx_kl            | 3.8304788e-05 |
|    clip_fraction        | 0.000244      |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0375       |
|    explained_variance   | 0             |
|    learning_rate        | 0.001 

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.07e+03      |
|    ep_rew_mean          | 676           |
| time/                   |               |
|    fps                  | 157           |
|    iterations           | 16            |
|    time_elapsed         | 1251          |
|    total_timesteps      | 3723264       |
| train/                  |               |
|    approx_kl            | 0.00013181679 |
|    clip_fraction        | 0.000382      |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0323       |
|    explained_variance   | 5.96e-08      |
|    learning_rate        | 0.001 

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.07e+03     |
|    ep_rew_mean          | 676          |
| time/                   |              |
|    fps                  | 155          |
|    iterations           | 23           |
|    time_elapsed         | 1820         |
|    total_timesteps      | 3809280      |
| train/                  |              |
|    approx_kl            | 1.551126e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.021       |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.001        |
|    l

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.07e+03     |
|    ep_rew_mean          | 677          |
| time/                   |              |
|    fps                  | 154          |
|    iterations           | 30           |
|    time_elapsed         | 2389         |
|    total_timesteps      | 3895296      |
| train/                  |              |
|    approx_kl            | 4.463537e-05 |
|    clip_fraction        | 0.000472     |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0288      |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    l

Collision occurred
Time limit exceeded
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.07e+03     |
|    ep_rew_mean          | 677          |
| time/                   |              |
|    fps                  | 153          |
|    iterations           | 37           |
|    time_elapsed         | 2957         |
|    total_timesteps      | 3981312      |
| train/                  |              |
|    approx_kl            | 8.063883e-05 |
|    clip_fraction        | 0.00133      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0307      |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    l

In [ ]:
'''
discrete_actions = {
    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}
Action 0: Steer left (-1) and throttle (1)
Action 1: Do not steer (0) and throttle (1)
Action 2: Steer right (1) and throttle (1)
Action 3: Do not steer (0) and do not throttle (0)
'''

In [ ]:
#discrete_actions = {
 #   0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}

In [ ]:
#len(discrete_actions)

In [4]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/three_throttle_one_steer/"
model_path = f"{models_dir}/4000000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

env = Monitor(env) 
evaluate_policy(model, env, n_eval_episodes=5,deterministic=True,return_episode_rewards= True)

Trying to spawn vehicle...
Vehicle spawned successfully!
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to target: 179.9302215576172 meters
 Throttle:1, Current speed: 0 km/h, Distance to targ

 Throttle:1, Current speed: 9 km/h, Distance to target: 179.6179962158203 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.59793090820312 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.5763397216797 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.5763397216797 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.5763397216797 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.5501251220703 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.5501251220703 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.52626037597656 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.52626037597656 meters
 Throttle:1, Current speed: 10 km/h, Distance to target: 179.52626037597656 meters
 Throttle:1, Current speed: 11 km/h, Distance to target: 179.49029541015625 meters
 Throttle:1, Current speed: 11 km/h, Distance to target: 179.49029541015625 meters
 Throttle:1

 Throttle:1, Current speed: 20 km/h, Distance to target: 177.82867431640625 meters
 Throttle:1, Current speed: 20 km/h, Distance to target: 177.82867431640625 meters
 Throttle:1, Current speed: 20 km/h, Distance to target: 177.78402709960938 meters
 Throttle:1, Current speed: 20 km/h, Distance to target: 177.78402709960938 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.7398681640625 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.7398681640625 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.69949340820312 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.69949340820312 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.6572265625 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.6144256591797 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.6144256591797 meters
 Throttle:1, Current speed: 21 km/h, Distance to target: 177.5682373046875 meters
 Throttle:1, 

 Throttle:1, Current speed: 30 km/h, Distance to target: 174.9718017578125 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.9718017578125 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.8951416015625 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.8951416015625 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.8951416015625 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.80029296875 meters
 Throttle:1, Current speed: 30 km/h, Distance to target: 174.80029296875 meters
 Throttle:1, Current speed: 31 km/h, Distance to target: 174.74154663085938 meters
 Throttle:1, Current speed: 31 km/h, Distance to target: 174.68441772460938 meters
 Throttle:1, Current speed: 31 km/h, Distance to target: 174.68441772460938 meters
 Throttle:1, Current speed: 31 km/h, Distance to target: 174.68441772460938 meters
 Throttle:1, Current speed: 31 km/h, Distance to target: 174.60836791992188 meters
 Throttle:1, Cu

 Throttle:1, Current speed: 44 km/h, Distance to target: 168.60433959960938 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.51683044433594 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.51683044433594 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.43121337890625 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.30247497558594 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.30247497558594 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.21224975585938 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.21224975585938 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.10684204101562 meters
 Throttle:1, Current speed: 45 km/h, Distance to target: 168.10684204101562 meters
 Throttle:1, Current speed: 46 km/h, Distance to target: 167.9723358154297 meters
 Throttle:1, Current speed: 46 km/h, Distance to target: 167.9723358154297 meters
 Throt

...

 Throttle:1, Current speed: 123 km/h, Distance to target: 82.3714370727539 meters
 Throttle:1, Current speed: 123 km/h, Distance to target: 82.17182159423828 meters
 Throttle:1, Current speed: 123 km/h, Distance to target: 81.9814682006836 meters
 Throttle:1, Current speed: 123 km/h, Distance to target: 81.79194641113281 meters
 Throttle:1, Current speed: 123 km/h, Distance to target: 81.79194641113281 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 81.48580932617188 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 81.24885559082031 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 81.24885559082031 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 81.028564453125 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 81.028564453125 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 80.79515075683594 meters
 Throttle:1, Current speed: 124 km/h, Distance to target: 80.79515075683594 meters
 Throttle:

 Throttle:1, Current speed: 134 km/h, Distance to target: 60.61641311645508 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 60.40854263305664 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 60.20508575439453 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 59.98960494995117 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 59.98960494995117 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 59.684600830078125 meters
 Throttle:1, Current speed: 134 km/h, Distance to target: 59.480072021484375 meters
 Throttle:1, Current speed: 135 km/h, Distance to target: 59.28017044067383 meters
 Throttle:1, Current speed: 135 km/h, Distance to target: 59.07575225830078 meters
 Throttle:1, Current speed: 135 km/h, Distance to target: 58.87287139892578 meters
 Throttle:1, Current speed: 135 km/h, Distance to target: 58.87287139892578 meters
 Throttle:1, Current speed: 135 km/h, Distance to target: 58.630245208740234 meters
 

 Throttle:1, Current speed: 144 km/h, Distance to target: 36.8862190246582 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 36.64286804199219 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 36.43416213989258 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 36.19771957397461 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 36.00149917602539 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 35.78823471069336 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 35.59457778930664 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 35.341304779052734 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 35.15250015258789 meters
 Throttle:1, Current speed: 144 km/h, Distance to target: 34.941829681396484 meters
 Throttle:1, Current speed: 145 km/h, Distance to target: 34.71718215942383 meters
 Throttle:1, Current speed: 145 km/h, Distance to target: 34.52383041381836 meters
 Th

 Throttle:1, Current speed: 152 km/h, Distance to target: 11.972638130187988 meters
 Throttle:1, Current speed: 152 km/h, Distance to target: 11.736141204833984 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 11.494537353515625 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 11.23162841796875 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 11.003622055053711 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 10.788265228271484 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 10.571797370910645 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 10.571797370910645 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 10.299997329711914 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 10.04987621307373 meters
 Throttle:1, Current speed: 153 km/h, Distance to target: 9.833635330200195 meters
Goal reached
Trying to spawn vehicle...
Vehicle spawned successfully!


([386.690786, 396.434248, 397.694709, 386.654755, 399.134233],
 [1685, 1667, 1678, 1658, 1681])